In [1]:
import pandas as pd
import numpy as np

In [28]:
df = pd.read_csv('C:/Users/KIHyuk/Documents/GitHub/Jupyter_Repo/Data/ab_data.csv')

In [ ]:
##https://www.kaggle.com/zhangluyuan/a-b-testing/notebook#Table-of-Contents

In [33]:
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [34]:
df.shape

(294478, 5)

In [35]:
## unique value 갯수
df.user_id.nunique() 

290584

In [38]:
##  category형으로 변환
df['converted'] = df['converted'].astype('category')

## category별 갯수 파악
df.groupby(df['converted']).size()

converted
0    259241
1     35237
dtype: int64

In [40]:
## 'group' 컬럼 파악
((df.group=='treatment') & (df.landing_page=='old_page')).sum()+ ((df.group=='control') & (df.landing_page=='new_page')).sum()

3893

In [50]:
## 결측치 파악

df.isnull().sum() # 컬럼별 결측치 갯수 반환

# df.notnull().sum()  # 컬럼별 결측값을 제외한 값의 개수 확인
# df.isnull().sum(1)  # row 별 결측값 개수 확인

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
dtype: int64

In [51]:
#identify misaligned rows
## ??? 데이터 오류 수정
df['misaligned']=((df.group=='treatment') & (df.landing_page=='old_page')) | ((df.group=='control') & (df.landing_page=='new_page'))

In [55]:
## 필터링을 위한 query 함수

df2 = df.query('misaligned == False')

df2.shape

(290585, 6)

In [56]:
# Double Check all of the correct rows were removed - this should be 0
# df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]

0

In [57]:
df2.nunique()

user_id         290584
timestamp       290585
group                2
landing_page         2
converted            2
misaligned           1
dtype: int64

In [62]:
# user_id별 빈도 수 파악

## value_counts() => 값이 몇개 나왔는가 #  df2.user_id.value_counts() => user_id 별 빈도수
df2.user_id.value_counts().sort_values(ascending=False).head()

773192    2
639032    1
663620    1
778364    1
645179    1
Name: user_id, dtype: int64

In [63]:
## 유일하게 2번 등장한 773192 id 파악하기

df2.query('user_id==773192')

,user_id,timestamp,group,landing_page,converted,misaligned
1899,773192,2017-01-09 05:37:58.781806,treatment,new_page,0,False
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0,False


In [64]:
## 최근 데이터 남기고 중복데이터 drop

df2.drop(1899,axis=0,inplace=True)

C:\Anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [66]:
## df2['converted'] 카테고리별 갯수 파악

df2.groupby(df2.converted).size()

converted
0    255831
1     34753
dtype: int64

In [72]:


actual_pold = (df2.query('group == "control"')['converted']==1).mean()
print(actual_pold)

actual_pnew=(df2.query('group=="treatment"')['converted']==1).mean()
print(actual_pnew)

(df2['landing_page']=='new_page').mean()

0.1203863045004612
0.11880806551510564


0.5000619442226688